# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [2]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.findAll("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 鬼魅之家, ID: 10459
Movie: 熱帶雨, ID: 10448
Movie: 吸血黏土, ID: 10439
Movie: 只想受到你的歡迎, ID: 10411
Movie: 大整形家, ID: 10395
Movie: 王牌辯士, ID: 10346
Movie: 紅鞋公主與七矮人, ID: 10417
Movie: 黑水風暴, ID: 10398
Movie: 1917, ID: 10189
Movie: 我們的7日戰爭, ID: 10453
Movie: 肥龍過江, ID: 10449
Movie: 殺手撿到槍, ID: 10423
Movie: 你的情歌, ID: 10407
Movie: 紳士追殺令, ID: 10317
Movie: 杜立德, ID: 10316
Movie: 喜從天降, ID: 10123
Movie: 深海終劫站, ID: 10230
Movie: 她們, ID: 10202
Movie: 變身特務, ID: 9008
Movie: 然後我們跳了舞, ID: 10451
Movie: 巧虎電影：英雄遊樂園, ID: 10429
Movie: 在黑暗中漫舞：20週年數位修復版, ID: 10428
Movie: 哥哥戀愛了，我怎麼辦！？, ID: 10414
Movie: 青春催落去, ID: 10404
Movie: 不完美的正義, ID: 10243
Movie: 絕地戰警FOR LIFE, ID: 10242
Movie: 惡靈20, ID: 10444
Movie: 重磅腥聞, ID: 10320
Movie: 真愛裁會贏, ID: 10362
Movie: 兔嘲男孩, ID: 10283
Movie: 海上鋼琴師4K數位修復, ID: 10425
Movie: 倒數計死, ID: 10390
Movie: 霹靂嬌娃, ID: 10124
Movie: 白頭山：半島浩劫, ID: 10354
Movie: 茱蒂, ID: 10112
Movie: 燃燒女子的畫像, ID: 10069
Movie: 葉問4：完結篇, ID: 9890
Movie: STAR WARS : 天行者的崛起, ID: 10225
Movie: 黑熊來了, ID: 10357
Movie: 特約經紀公司, ID: 10364


### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [3]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10189

In [4]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
}
    
# resp = requests.get(url, params=payload)
resp = requests.get(url, params=payload, headers=headers)
# print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [5]:
# 指定放映地區
area_id = 28

In [6]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_.")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

二月 5
二月 6
二月 7
二月 8
二月 9


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [7]:
# 選定要觀看的日期
date = "2020-02-06"

In [8]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id)}

resp = requests.get(url, params=payload)
print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

f6ab8e"
                        data-date="1580918400" data-time="1580987400"
                        data-movie_type="數位"
                        data-movie_title="1917"
                        data-movie_date="02.06"
                        data-movie_time="19:10"
                        data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '1917']" data-ezding="ezding"
                        />
                    <label for="72882741" class="select">19:10</label>
                                               </div>
           </li>
          </ul>
      <ul id="theater_id_144" class="area_time _c jq_area_time" data-theater_name="台北新民生戲院" data-theater_url="http://www.newminshen.com/" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=144" >
       <li class="adds">
           <a href="https://movies.yahoo.com.tw/theater_result.html/id=144">台北新民生戲院</a>
           <span>02-27653373</span>
       </li>
                         <li class="taps">
                <span class="tapR"

In [9]:
# print(html)
for i in html:
  t = i.find(class_="tapR").string
  times = i.findAll(class_="gabtn")
  print(i["data-theater_name"])
  print(t)
  for time in times:
    print(time["data-movie_time"]) 
  

台北真善美劇院
數位
12:40
14:40
19:00
喜滿客絕色影城
數位
10:30
14:45
17:00
19:15
21:30
22:45
23:45
梅花數位影院
數位
10:00
14:30
19:10
台北新民生戲院
數位
10:40
14:45
19:00
喜樂時代影城南港店
數位
10:20
11:30
12:40
13:50
15:00
16:10
17:20
18:00
18:35
19:35
20:15
20:55
21:50
22:30
23:15
